# INTELIGENCIA ARTIFICIAL (INF 371)
Dr. Edwin Villanueva

# Practica Guiada 1:  Entorno de Laberinto y Agente Reactivo

En este notebook vamos a practicar la implementacion de un entorno de un laberinto y habilitar un agente reactivo en el mismo. Al final de este notebook estan las preguntas que deben ser respondidas. 

El laberinto es un leido de un archivo de texto que puede terner los siguientes caracteres:

    '#': Indica que es una celda con obstaculo  (impasable) 
    '~': Indica que es una celda con agua (pasable, con costo 5)
    ' ': Indica que es una celda vacia (passable con costo 1)
    'S': Indica que es celda de Inicio
    'G': Indicates que es celda Objetivo ( premio de 100 )
    
 


# Funciones para leer el laberinto de disco

<b>readMazeFromFile</b>  La funcion readMazeFromFile lee un archivo de disco que contiene un laberinto y retorna una matriz de celdas del laberinto leido (grid).    grid es una lista de listas, ejemplo: [['#','S',' '],['#',' ','G'],['~','#','~']]

In [97]:
def readMazeFromFile(file):
    fin = open(file)
    lines = fin.read().splitlines()
    grid = []
    for line in lines: 
        grid.append(list(line))
    return grid    # grid es una lista de lista

<b>getLocCells:</b> La funcion getLocCells devuelve la localizacion de la celda inicial (S) y celda objetivo (G) en un grid

In [98]:
def getLocCells(grid):
    numRows = len(grid)
    numCols = len(grid[0])
    for i in range(numRows):
        for j in range(numCols):
            if len(grid[i]) != numCols:
                raise "Grid no Rectangular"
            if grid[i][j] == 'S':
                cell_S = (i,j)
            if grid[i][j] == 'G':
                cell_G = (i,j)
                
    if cell_S == None:
        raise "No hay celda de Inicio"
    if cell_G == None:
        raise "No hay celda Objetivo (G)"
        
    return cell_S, cell_G

#### <b> Funciones para mostrar los resultados en el laberinto</b> 

In [99]:
def grid2Str(grid):
    """ Convierte un grid a un string para vizualizacion """
    numRows = len(grid)
    numCols = len(grid[0])
    strGrid = []
    headerLine = ' ' + ('-' * (numCols)) + ' '
    strGrid.append(headerLine)
    for row in grid:
        rowLine = '|' + ''.join(row) + '|'
        strGrid.append(rowLine)
    strGrid.append(headerLine)
    return '\n'.join(strGrid)

In [100]:
def showAgent(grid, agent_location, agent_performance):
    """ Muestra los resultados en el grid.   """
    grid_copy = []
    for row in grid:
        grid_copy.append([x for x in row]) 
    row,col = agent_location
    grid_copy[row][col] = 'X' 
    print (grid2Str(grid_copy))
    print ('Desempeño del agente = {}'.format(agent_performance))
    print ("-------------------------------")

# Implementacion del Entorno  


#### Clase <b>Thing</b>

  Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [101]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

#### Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [102]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        raise NotImplementedError

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        
        if location is not None:
            thing.location = location 
        else:
            #print('colocara localizacion por defecto')
            self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>MazeEnvironment</b>

Esta clase implementa el entorno del laberinto. El estado percibido por un agente en este entorno corresponde a la tupla [grid, location]. Las acciones posibles para un agente son: 'N' (ir una celda al norte), 'S' (ir una celda al sur), 'W' (ir una celda al oeste), 'E' (ir una celda al este)  

In [103]:
class MazeEnvironment(Environment):
    
    def __init__(self, grid):
        super().__init__()
        self.grid = grid
        self.numRows = len(grid)
        self.numCols = len(grid[0])
        self.startCell, self.goalCell = getLocCells(grid)
        
    def default_location(self, agent):
        """Coloca Localización por defecto a un agente sin localizacion especificada (en celda 'S')."""
        agent.location = self.startCell
        
    def thing_classes(self):
        return [Agent]

    def percept(self, agent):
        """Retorna el estado del ambiente (location)"""
        return agent.location
    
    def __isValidLocation(self,location):
        """ Retorna true si la localizacion dada corresponde a una celda no bloqueada valida """
        row,col = location
        if row < 0 or row >= self.numRows:
            return False
        if col < 0 or col >= self.numCols:
            return False
        return not self.grid[row][col] == '#'       

    def execute_action(self, agent, action):
        """Implementa el MAPA De TRANSICION: Cambia la posicion del agente de acuerdo a la accion solicitada """
        row,col = agent.location   # obtiene posicion actual del agente
        
        if action == 'N' and self.__isValidLocation((row-1, col)):
            agent.location = (row-1, col)
            agent.performance -= 1
            if self.grid[row-1][col]== '~':
                agent.performance -= 5
            
        elif action == 'S' and self.__isValidLocation((row+1, col)):
            agent.location = (row+1, col)
            agent.performance -= 1  
            if self.grid[row+1][col]== '~':
                agent.performance -= 5
            
        elif action == 'W' and self.__isValidLocation((row, col-1)):
            agent.location = (row, col-1)
            agent.performance -= 1
            if self.grid[row][col-1]== '~':
                agent.performance -= 5
            
        elif action == 'E' and self.__isValidLocation((row, col+1)):
            agent.location = (row, col+1)
            agent.performance -= 1
            if self.grid[row][col+1]== '~':
                agent.performance -= 5
                        
        if agent.location == self.goalCell: 
            agent.performance += 100     # suma 100 puntos al desempeño del agente cuando llega a celda objetivo 
        
        
        



# Implementacion de un Agente Reactivo  

#### Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [104]:
import collections
import random

class Agent(Thing):
    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

#### <b>Clase que implementa un Programa de Agente Reactivo para el entorno MazeEnvironment</b>


In [105]:
import random
class MazeReactiveAgentProgram:
    def __init__(self, grid):
        self.grid = grid
        
    def __call__(self, percept):
        state = percept
        print('Agente recibiendo percepcion de localizacion = {}'.format(state))
        action = '' 
        actionList = ['N','W','E']
        
        if state != (0,0):
            
            if state[0] == 0: # si estamos en la fila  0 no se necesitará ir al sur ( caso más simple )
                actionList.remove('N')
                
            if state[1] == 0: # si estamos en la columna 0
                actionList.remove('W')  
                
            action = random.choice(actionList)
            
            
#         if state == (2,4):
#             action = 'N'
#         if state == (1,4):
#             action = 'N'
#         if state == (0,4):
#             action = 'W'
#         if state == (0,3):
#             action = 'W'
#         if state == (0,2):
#             action = 'W'
#         if state == (0,1):
#             action = 'W'
        
        return action 

# Probando el agente en MazeEnvironment

In [106]:
# Carga un laberinto de archivo en disco, instancia el entorno y visualiza el grid
mazegrid = readMazeFromFile('maze.txt') 
e = MazeEnvironment(mazegrid)
print(grid2Str(mazegrid))


 ------------ 
|G       ~~~#|
|####~~~~~~~#|
|~   S    ~~#|
|## # #####~#|
|#  #      ~#|
|# ##### ####|
|#       ~~~#|
|## ### #####|
 ------------ 


In [107]:
# Crea un agente reactivo y lo añade al entorno del laberinto
a = Agent( MazeReactiveAgentProgram (mazegrid) ) 
e.add_thing(a) 

showAgent (mazegrid, a.location, a.performance)   # Agente es mostrado como una X


 ------------ 
|G       ~~~#|
|####~~~~~~~#|
|~   X    ~~#|
|## # #####~#|
|#  #      ~#|
|# ##### ####|
|#       ~~~#|
|## ### #####|
 ------------ 
Desempeño del agente = 0
-------------------------------


In [121]:
# Ejecuta 1 pasos del ambiente 
e.run(1)
showAgent (mazegrid, a.location, a.performance)  # Agente es mostrado como una X


Agente recibiendo percepcion de localizacion = (0, 1)
 ------------ 
|X       ~~~#|
|####~~~~~~~#|
|~   S    ~~#|
|## # #####~#|
|#  #      ~#|
|# ##### ####|
|#       ~~~#|
|## ### #####|
 ------------ 
Desempeño del agente = 76
-------------------------------


# Preguntas:

1) <b>Implemente correctamente la medida de desempeño segun los costos de las celdas dados al inicio de este notebook<b>


2) <b>Reimplemente el programa reactivo para exhibr una mejor racionalidad en un  entorno general de laberinto<b>

In [ ]:
La idea era analizar el caso más simple cuando el goal esté en la esquina para luego poder hacer un caso más general 
pero ya no pude implementarlo.

# if state != (0,0):
            
#             if state[0] == 0: # si estamos en la fila  0 no se necesitará ir al sur ( caso más simple )
#                 actionList.remove('N')
                
#             if state[1] == 0: # si estamos en la columna 0
#                 actionList.remove('W')  
                
#             action = random.choice(actionList)

 
3) <b>Explique porqué un agente de resolucion de problemas seria mas racional en este entorno<b> 

4) <b>Qué capacidades deberia tener el agente si el entorno es desconocido (no tiene acceso al grid)? (asumo que conoce el exit)

El agente debe tener racionalidad para poder optimizar el desempeño de la busqueda porque debe de encontrar el camino 
más corto; debe de tener la capacidad de colectar información dado que no se conoce el laberinto ; y ,por último,
debe de tener la capacidad de aprender pues en caso de que se tope con un camino sin salida no deberia de volver a este.